### 予測モデルの仕組みをわかりやすく説明します！

#### どんなことを予測するの？
このモデルは、**「当月末時点での翌月のスタートMP」**と**「当月末時点での翌月末MP」**を予測するものです。

#### 具体例で説明します

**例：2月12日時点のケース**

最新データが2/12の場合、このモデルは以下のように動作します：

**ステップ1: 2月12日時点の「次月（3月）データ」を確認**
- 次月確定MP：500,000円
- 次月A：45,000,000円
- 次月B：4,500,000円
- 次月C：3,000,000円
- 次月D：15,000,000円

**ステップ2: 2月末時点での3月スタートMPを予測**
- 3月スタートMP = 次月確定 + 次月A
- 500,000円 + 45,000,000円 = **45,500,000円**
- これは「2月末に確定する3月のスタート値」です

**ステップ3: 3月末のMPを予測**
- 3月スタートMP + 3月中の追加分 = 3月末MP
- 45,500,000円 + 追加分 = **約68,000,000円（予測）**

---

**別の例：4月15日時点のケース**

**ステップ1: 4月15日時点の「次月（5月）データ」を確認**
- 次月確定MP：500,000円
- 次月A：45,000,000円
- 次月B、C、D：各種データ

**ステップ2: 4月末時点での5月スタートMPを予測**
- 5月スタートMP = 次月確定 + 次月A = **45,500,000円**
- これは「4月末に確定する5月のスタート値」です

**ステップ3: 5月末のMPを予測**
- 5月スタートMP + 5月中の追加分 = **約68,000,000円（予測）**

---

#### なぜこのモデルが便利なの？

1. **当月中に、翌月の見通しが立てられる**
   - 2月12日時点で、既に「3月はこれくらいのMPでスタートできる」が分かる
   - さらに「3月末にはこれくらいまで伸びそうだ」も予測できる

2. **日ごとに予測が更新される**
   - 2月1日、2月15日、2月28日と、日が進むにつれて予測の精度が上がる
   - 次月のデータが積み上がるほど、翌月の見通しがクリアになる

#### どうやって予測するの？

1. **翌月のスタートMP（確定部分）**
   - 次月確定MP + 次月A = スタートMP
   - これは計算するだけ（予測不要）

2. **翌月末までの追加分（予測部分）**
   - 次月B、C、Dが翌月中にどれくらい追加MPに変わるか
   - 過去のデータから学んだ「転換率」を使って予測
   - ランダムフォレストと月別転換率を組み合わせて精度向上

3. **合計**
   - スタートMP + 追加分 = 翌月末の予測MP

#### まとめ

**このモデルは、当月の日次データ（次月確定、次月A、次月B、次月C、次月D）を使って**：
- ✅ **当月末時点で確定する翌月のスタートMP**（次月確定 + 次月A）
- ✅ **当月末時点での翌月末MPの予測**

**この2つを教えてくれる予測ツールです！**

# MP Next Month-End Prediction Model (as of Current Month-End)
## 当月末時点での翌月末MP予測モデル

## 1. ライブラリのインポート

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import warnings
warnings.filterwarnings('ignore')

print('Libraries loaded!')

## 2. データの読み込み

In [ ]:
try:
    from google.colab import files
    print('Upload CSV file:')
    uploaded = files.upload()
    filename = list(uploaded.keys())[0]
except:
    filename = 'progress_data.csv'
    print(f'Using: {filename}')

In [ ]:
try:
    df = pd.read_csv(filename, encoding='utf-8-sig')
except:
    df = pd.read_csv(filename, encoding='shift_jis')

print(f'Shape: {df.shape}')
print(f'Columns: {df.columns.tolist()}')
df.head()

## 3. データ前処理（日次の次月データを使用）

In [ ]:
# 元のカラム名を保存
original_columns = df.columns.tolist()
print(f'Original columns: {original_columns}')

# 日付処理
df['date'] = pd.to_datetime(df.iloc[:, 0], errors='coerce')
# NaT（無効な日付）を含む行を削除
df = df.dropna(subset=['date']).copy()

# 年月情報を追加
df['year_month'] = df['date'].dt.to_period('M')
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day

print(f'\nDaily data points: {len(df)}')

In [ ]:
# カラム名を変更（次月の確定、A、B、C、Dに対応）
# 13列フォーマットの場合は列8-12、12列フォーマットの場合は列7-11、7列フォーマットの場合は列2-6を使用
if len(original_columns) >= 13:
    # 13列フォーマット: 次月データを使用（列8-12）
    col_mapping = {
        original_columns[1]: 'Current_MP',     # 当月MP（参考用）
        original_columns[7]: 'Next_MP',        # 次月MP（参考用）
        original_columns[8]: 'Next_Kakutei',   # 次月確定MP
        original_columns[9]: 'Next_A',         # 次月AMP
        original_columns[10]: 'Next_B',        # 次月BMP
        original_columns[11]: 'Next_C',        # 次月CMP
        original_columns[12]: 'Next_D'         # 次月DMP
    }
    print('Using 13-column format (next month data from columns 8-12)')
elif len(original_columns) >= 12:
    # 12列フォーマット: 次月データを使用（列7-11）
    col_mapping = {
        original_columns[1]: 'Current_MP',     # 当月MP（参考用）
        original_columns[7]: 'Next_Kakutei',   # 次月確定MP
        original_columns[8]: 'Next_A',         # 次月AMP
        original_columns[9]: 'Next_B',         # 次月BMP
        original_columns[10]: 'Next_C',        # 次月CMP
        original_columns[11]: 'Next_D'         # 次月DMP
    }
    print('Using 12-column format (next month data from columns 7-11)')
else:
    # 7列フォーマット: 次月データのみ（列2-6）
    col_mapping = {
        original_columns[1]: 'Current_MP',
        original_columns[2]: 'Next_Kakutei',
        original_columns[3]: 'Next_A',
        original_columns[4]: 'Next_B',
        original_columns[5]: 'Next_C',
        original_columns[6]: 'Next_D'
    }
    print('Using 7-column format (next month data only)')

print(f'Mapping: {col_mapping}')

df = df.rename(columns=col_mapping)

# 数値変換
for col in ['Current_MP', 'Next_MP', 'Next_Kakutei', 'Next_A', 'Next_B', 'Next_C', 'Next_D']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

print(f'\nColumns after mapping: {df.columns.tolist()}')
df.head()

In [ ]:
# 各月の翌月のMPを取得（ターゲット変数として使用）
# データの流れの例：
# - 2月12日のデータには「次月（3月）データ」が含まれる
# - 2月末時点で3月のスタートMP（次月確定+次月A）が確定
# - 3月末のMPは、3月の月末データから取得される
# このモデルは、2月の各日から「3月末のMP」を予測する

# 翌月のyear_monthを計算
df['next_year_month'] = (df['year_month'] + 1)

# 翌月末のMPを取得
next_month_end_mp = df.groupby('year_month')['Current_MP'].last().to_dict()
df['Next_Month_End_MP'] = df['next_year_month'].map(next_month_end_mp)

# 月末日を特定
month_end_dates = df.groupby('year_month')['date'].max()
df['is_month_end'] = df.apply(lambda x: x['date'] == month_end_dates[x['year_month']], axis=1)

# 訓練データ作成（月末以外の日のデータを使用）
train_data = df[~df['is_month_end'] & df['Next_Month_End_MP'].notna()].copy()

print(f'Training data points: {len(train_data)} days')
print(f'Unique months: {train_data["year_month"].nunique()}')

if len(train_data) > 0:
    # 次月のスタートMP（確定部分）= 当月末時点で確定する翌月のスタート値
    train_data['Next_Month_Start_MP'] = train_data['Next_Kakutei'] + train_data['Next_A']
    
    # 次月に追加で増える部分 = 翌月中にBCDから変換される追加分
    train_data['Additional'] = train_data['Next_Month_End_MP'] - train_data['Next_Month_Start_MP']
    
    # 次月のBCD合計
    train_data['Next_BCD_Total'] = train_data['Next_B'] + train_data['Next_C'] + train_data['Next_D']
    
    # 転換率計算
    train_data['Conversion_Rate'] = np.where(
        train_data['Next_BCD_Total'] > 0,
        train_data['Additional'] / train_data['Next_BCD_Total'] * 100,
        0
    )
    
    # 予測対象の月を計算
    train_data['target_month'] = (train_data['month'] % 12) + 1
    
    print('Calculations complete!')
    print(train_data[['date', 'year_month', 'target_month', 'Next_Month_Start_MP', 'Next_BCD_Total', 'Next_Month_End_MP']].head(10))
else:
    print('ERROR: No training data!')

## 4. 月別転換率の分析

In [ ]:
# 予測対象月別の転換率
monthly_stats = train_data.groupby('target_month').agg({
    'Conversion_Rate': ['mean', 'std', 'count']
}).round(1)
monthly_stats.columns = ['Avg_Rate', 'Std', 'Count']

print('Monthly BCD Conversion Rate (Target Month)')
print('='*50)
month_names = ['', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
               'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
for m in range(1, 13):
    if m in monthly_stats.index:
        r = monthly_stats.loc[m]
        print(f'{m:2d} ({month_names[m]}): {r["Avg_Rate"]:5.1f}% (n={int(r["Count"])})')
        
display(monthly_stats)

In [ ]:
# 月内の日による転換率の変化を可視化
fig, axes = plt.subplots(3, 4, figsize=(16, 10))
axes = axes.flatten()

for m in range(1, 13):
    month_data = train_data[train_data['target_month'] == m]
    if len(month_data) > 0:
        daily_avg = month_data.groupby('day')['Conversion_Rate'].mean()
        axes[m-1].plot(daily_avg.index, daily_avg.values, marker='o')
        axes[m-1].set_title(f'{month_names[m]} - Conversion Rate by Day')
        axes[m-1].set_xlabel('Day of Current Month')
        axes[m-1].set_ylabel('Rate (%)')
        axes[m-1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. モデル構築

In [ ]:
class NextMonthMPPredictor:
    def __init__(self):
        self.model = None
        self.monthly_rates = {}
        self.overall_avg_rate = None

    def train(self, data):
        # 予測対象月別の平均転換率を計算
        for m in range(1, 13):
            sub = data[data['target_month'] == m]
            if len(sub) > 0:
                self.monthly_rates[m] = sub['Conversion_Rate'].mean()
        self.overall_avg_rate = data['Conversion_Rate'].mean()

        # Random Forestモデルの訓練
        X = data[['Next_B', 'Next_C', 'Next_D', 'target_month', 'day']]
        y = data['Additional']
        
        self.model = RandomForestRegressor(
            n_estimators=100, 
            max_depth=7,
            random_state=42
        )
        self.model.fit(X, y)

        # モデル評価
        pred_additional = self.model.predict(X)
        pred = data['Next_Month_Start_MP'] + pred_additional
        
        return {
            'mae': mean_absolute_error(data['Next_Month_End_MP'], pred),
            'r2': r2_score(data['Next_Month_End_MP'], pred)
        }

    def predict(self, next_kakutei, next_a, next_b, next_c, next_d, current_month, current_day):
        """日次の次月データから翌月末のMPを予測"""
        # 翌月のスタートMP
        next_month_start = next_kakutei + next_a
        next_bcd = next_b + next_c + next_d
        
        # 予測対象月（翌月）
        target_month = (current_month % 12) + 1

        # Random Forestによる予測
        add_rf = self.model.predict([[next_b, next_c, next_d, target_month, current_day]])[0]
        
        # 月別転換率による予測
        rate = self.monthly_rates.get(target_month, self.overall_avg_rate) / 100
        add_rate = next_bcd * rate
        
        # ハイブリッド予測
        additional = add_rf * 0.7 + add_rate * 0.3

        return {
            'current_month': current_month,
            'current_day': current_day,
            'target_month': target_month,
            'next_month_start': next_month_start,
            'additional': additional,
            'forecast': next_month_start + additional,
            'next_bcd': next_bcd,
            'rate': rate * 100
        }

print('NextMonthMPPredictor defined!')

In [ ]:
predictor = NextMonthMPPredictor()
metrics = predictor.train(train_data)

print('Model Trained!')
print(f'MAE: {metrics["mae"]/10000:,.0f} (10K JPY)')
print(f'R2:  {metrics["r2"]:.3f}')

## 6. 予測実行

In [ ]:
def show_prediction(p):
    mn = ['','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
    print('='*70)
    print(f'📅 Prediction Date: {mn[p["current_month"]]} {p["current_day"]}')
    print(f'🎯 Target Month: {mn[p["target_month"]]}')
    print('='*70)
    print(f'\n💰 {mn[p["target_month"]]} Start MP (at {mn[p["current_month"]]} end):')
    print(f'   Next Kakutei + Next A = {p["next_month_start"]/10000:>10,.0f} (10K JPY)')
    print(f'   ↑ This is the guaranteed MP for {mn[p["target_month"]]} start')
    print(f'\n📈 Additional MP (predicted growth during {mn[p["target_month"]]}):')
    print(f'   From B, C, D conversion = {p["additional"]/10000:>10,.0f} (10K JPY)')
    print(f'   Conversion Rate: {p["rate"]:.1f}%')
    print(f'\n🎯 {mn[p["target_month"]]} END FORECAST:')
    print(f'   Start MP + Additional = {p["forecast"]/10000:>10,.0f} (10K JPY)')
    print('='*70)

In [ ]:
# 最新データで予測
latest = df[~df['is_month_end'] & df['Next_Month_End_MP'].notna()].iloc[-1]
print(f'Latest: {latest["date"].strftime("%Y-%m-%d")}')
print(f'Current Month: {latest["year_month"]}')
print(f'Predicting for: {latest["next_year_month"]}')
print()

p = predictor.predict(
    latest['Next_Kakutei'], latest['Next_A'],
    latest['Next_B'], latest['Next_C'], latest['Next_D'],
    current_month=int(latest['month']),
    current_day=int(latest['day'])
)
show_prediction(p)

# 実際の翌月末MPと比較
if pd.notna(latest['Next_Month_End_MP']):
    actual = latest['Next_Month_End_MP']
    print(f'\nActual Next Month-End MP: {actual/10000:>10,.0f} (10K)')
    print(f'Prediction Error: {abs(p["forecast"] - actual)/10000:>10,.0f} (10K)')

In [ ]:
# カスタム予測例
# 例: 2月12日時点の次月データから3月末を予測
# 2月末に3月のスタートMPがいくらになるか、3月末にMPがいくらになるかを予測
p = predictor.predict(
    next_kakutei=500000,
    next_a=45000000,
    next_b=4500000,
    next_c=3000000,
    next_d=15000000,
    current_month=2,  # 2月
    current_day=12    # 12日
)
show_prediction(p)

## 7. 月内での翌月予測の推移

In [ ]:
# 最新月のデータで、日ごとに翌月末の予測がどう変化するかシミュレーション
latest_month = df[(df['year_month'] == df['year_month'].max()) & ~df['is_month_end'] & df['Next_Month_End_MP'].notna()].copy()

if len(latest_month) > 0:
    simulation_results = []
    
    for _, row in latest_month.iterrows():
        p = predictor.predict(
            row['Next_Kakutei'], row['Next_A'],
            row['Next_B'], row['Next_C'], row['Next_D'],
            current_month=int(row['month']),
            current_day=int(row['day'])
        )
        simulation_results.append({
            'date': row['date'],
            'day': row['day'],
            'next_month_start': p['next_month_start'],
            'additional': p['additional'],
            'forecast': p['forecast'],
            'actual': row['Next_Month_End_MP']
        })
    
    sim_df = pd.DataFrame(simulation_results)
    
    current_ym = latest_month.iloc[0]['year_month']
    next_ym = latest_month.iloc[0]['next_year_month']
    
    # グラフ1: 翌月末予測の推移
    plt.figure(figsize=(14, 6))
    plt.plot(sim_df['day'], sim_df['forecast']/10000, 
             label='Predicted Next Month-End MP', marker='o', linewidth=2, color='steelblue')
    plt.axhline(y=sim_df['actual'].iloc[0]/10000, 
                color='red', linestyle='--', label='Actual Next Month-End MP', linewidth=2)
    plt.fill_between(sim_df['day'], 
                     sim_df['next_month_start']/10000, 
                     sim_df['forecast']/10000, 
                     alpha=0.3, label='Additional (Predicted)', color='lightblue')
    
    plt.title(f'Daily Prediction Progress: {current_ym} → {next_ym} End')
    plt.xlabel(f'Day of {current_ym}')
    plt.ylabel('MP (10K JPY)')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    print(f'\nDaily prediction for {next_ym} End from each day of {current_ym}:')
    display(sim_df[['day', 'next_month_start', 'additional', 'forecast', 'actual']].head(10))
else:
    print('No data available for simulation')
    print(f'Latest month in data: {df["year_month"].max()}')
    print(f'Months with next month end MP: {df[df["Next_Month_End_MP"].notna()]["year_month"].unique()}')

## 8. 翌月スタートMPの推移分析

### 必要なデータ：
セクション8が動作するには、以下のデータが必要です：

1. **最新月の日次データ**（月末を除く）
   - 例：最新が2/12の場合、2月の1日～11日のデータ

2. **次月データ**（次月確定、次月A、次月B、次月C、次月D）
   - 各日付に次月データが記録されていること

3. **翌月末のMP**（Next_Month_End_MP）
   - 翌月の実績値が必要（過去データの場合）
   - または、最新月の場合は予測のみ表示

### データ診断セル（次のセルで確認）：

In [ ]:
# 当月の各日における翌月スタートMP（次月確定+次月A）の推移
# セクション7で作成されたデータを使用
latest_month_for_start = df[(df['year_month'] == df['year_month'].max()) & ~df['is_month_end'] & df['Next_Month_End_MP'].notna()].copy()

if len(latest_month_for_start) > 0:
    # 日ごとの翌月スタートMPを計算
    start_mp_results = []
    
    for _, row in latest_month_for_start.iterrows():
        start_mp_results.append({
            'date': row['date'],
            'day': row['day'],
            'next_month_start': row['Next_Kakutei'] + row['Next_A']
        })
    
    start_mp_df = pd.DataFrame(start_mp_results)
    
    current_ym = latest_month_for_start.iloc[0]['year_month']
    next_ym = latest_month_for_start.iloc[0]['next_year_month']
    
    # グラフ2: 翌月スタートMPの推移
    plt.figure(figsize=(14, 6))
    plt.plot(start_mp_df['day'], start_mp_df['next_month_start']/10000, 
             marker='o', linewidth=2, color='green', label=f'{next_ym} Start MP (Kakutei + A)')
    
    # 最終値を強調
    final_start_mp = start_mp_df['next_month_start'].iloc[-1]
    plt.axhline(y=final_start_mp/10000, 
                color='darkgreen', linestyle='--', alpha=0.5, 
                label=f'{current_ym} End → {next_ym} Start MP')
    
    plt.title(f'Next Month Start MP Progress by Day: {current_ym} → {next_ym} Start')
    plt.xlabel(f'Day of {current_ym}')
    plt.ylabel('Next Month Start MP (10K JPY)')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    print(f'\n{next_ym} Start MP Statistics:')
    print('='*60)
    print(f'Final ({current_ym} end):  {start_mp_df["next_month_start"].iloc[-1]/10000:>10,.0f} (10K JPY)')
    print(f'Average:                    {start_mp_df["next_month_start"].mean()/10000:>10,.0f} (10K JPY)')
    print(f'Maximum:                    {start_mp_df["next_month_start"].max()/10000:>10,.0f} (10K JPY)')
    print(f'Minimum:                    {start_mp_df["next_month_start"].min()/10000:>10,.0f} (10K JPY)')
    print('='*60)
    print(f'\n💡 Interpretation:')
    print(f'   - By {current_ym} end, {next_ym} will start with {final_start_mp/10000:,.0f} (10K JPY)')
    print(f'   - This is the guaranteed MP (Next Kakutei + Next A) for {next_ym}')
else:
    print('No data available for next month start MP analysis')
    print(f'Latest month in data: {df["year_month"].max()}')
    print(f'Months with next month end MP: {df[df["Next_Month_End_MP"].notna()]["year_month"].unique()}')

In [ ]:
# データ診断：セクション8に必要なデータの確認
print('='*70)
print('DATA DIAGNOSIS FOR SECTION 8')
print('='*70)

# 1. 最新月の確認
print(f'\n1. Latest month in data: {df["year_month"].max()}')
print(f'   Latest date: {df["date"].max().strftime("%Y-%m-%d")}')

# 2. 最新月の日次データ確認
latest_month_data = df[df['year_month'] == df['year_month'].max()]
print(f'\n2. Latest month daily data:')
print(f'   Total days: {len(latest_month_data)}')
print(f'   Non-month-end days: {len(latest_month_data[~latest_month_data["is_month_end"]])}')
print(f'   Date range: {latest_month_data["date"].min().strftime("%Y-%m-%d")} to {latest_month_data["date"].max().strftime("%Y-%m-%d")}')

# 3. 次月データの確認
print(f'\n3. Next month data availability:')
for col in ['Next_Kakutei', 'Next_A', 'Next_B', 'Next_C', 'Next_D']:
    if col in df.columns:
        non_null = latest_month_data[col].notna().sum()
        print(f'   {col}: {non_null}/{len(latest_month_data)} days have data')

# 4. Next_Month_End_MP の確認
print(f'\n4. Next_Month_End_MP availability:')
print(f'   Total records with Next_Month_End_MP: {df["Next_Month_End_MP"].notna().sum()}')
print(f'   Months with Next_Month_End_MP:')
months_with_data = df[df["Next_Month_End_MP"].notna()]["year_month"].unique()
for ym in sorted(months_with_data)[-5:]:  # 直近5ヶ月のみ表示
    print(f'     - {ym}')

# 5. セクション8の条件チェック
print(f'\n5. Section 8 condition check:')
condition1 = df['year_month'] == df['year_month'].max()
condition2 = ~df['is_month_end']
condition3 = df['Next_Month_End_MP'].notna()
combined = condition1 & condition2 & condition3

print(f'   Latest month: {condition1.sum()} records')
print(f'   Non-month-end: {condition2.sum()} records')  
print(f'   Has Next_Month_End_MP: {condition3.sum()} records')
print(f'   ALL conditions met: {combined.sum()} records')

print(f'\n6. Recommendation:')
if combined.sum() > 0:
    print(f'   ✅ Section 8 should work! ({combined.sum()} days available)')
else:
    print(f'   ⚠️  Section 8 will not display (no matching data)')
    print(f'   ')
    print(f'   This is expected for the current month (e.g., Feb 2026)')
    print(f'   because Next_Month_End_MP (Mar 2026 end) doesn\'t exist yet.')
    print(f'   ')
    print(f'   To see Section 8 visualization, use data from a past month')
    print(f'   that has complete next month data.')

print('='*70)